<div style="max-width:1400px;margin-center: auto">
<img src="images\pitstop.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("pitstop") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [3]:
pit_stops_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("stop", StringType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("duration", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])


In [4]:
pit_stops_df = spark.read.json(r"E:\unused\Udemy\Spark_practice\raw\raw_files\pit_stops.json", 
                             schema = pit_stops_schema, 
                            multiLine = True)
pit_stops_df.show(2)

+------+--------+----+---+--------+--------+------------+
|raceId|driverId|stop|lap|    time|duration|milliseconds|
+------+--------+----+---+--------+--------+------------+
|   841|     153|   1|  1|17:05:23|  26.898|       26898|
|   841|      30|   1|  1|17:05:52|  25.021|       25021|
+------+--------+----+---+--------+--------+------------+
only showing top 2 rows



In [5]:
from pyspark.sql.functions import current_timestamp

In [6]:
final_df = pit_stops_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumn("ingestion_date", current_timestamp())

In [7]:
final_df.write.mode('overwrite').format('parquet').saveAsTable('f1_processed.pitstop')